<a href="https://colab.research.google.com/github/JongHyun2332/r1/blob/master/20211007%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 불러오기 및 기본 설정**

In [ ]:
#기본설정
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
color = sns.color_palette()
import warnings
warnings.filterwarnings('ignore')
# Garbage Collector to free up memory
import gc                         
gc.enable() 

In [ ]:
#데이터 불러오기
orders = pd.read_csv('orders.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

In [ ]:
#데이터 크기 줄이기
def int_memory_reduce(data) :
    data_int = data.select_dtypes(include=['int'])
    converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')
    print(f"Before : {mem_usage(data_int)} -> After : {mem_usage(converted_int)}")
    data[converted_int.columns] = converted_int
    return data

In [ ]:
def float_memory_reduce(data) :
    data_float = data.select_dtypes(include=['float'])
    converted_float = data_float.apply(pd.to_numeric,downcast='float')
    print(f"Before : {mem_usage(data_float)} -> After : {mem_usage(converted_float)}")
    data[converted_float.columns] = converted_float
    return data

def object_memory_reduce(data) :
    gl_obj = data.select_dtypes(include=['object']).copy()
    converted_obj = pd.DataFrame()
    for col in gl_obj.columns:
        num_unique_values = len(gl_obj[col].unique())
        num_total_values = len(gl_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = gl_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = gl_obj[col]
    print(f"Before : {mem_usage(gl_obj)} -> After : {mem_usage(converted_obj)}")
    data[converted_obj.columns] = converted_obj
    return data

In [ ]:
#데이터 확인
#orders.head()
#order_products_prior.head()
#order_products_train.head()
#products.head()
#aisles.head()
#departments.head()
#order_products_train.shape
#order_products_train.isnull().sum()

# **데이터 양 줄이기**
데이터 크기는 줄였으나 워낙 데이터가 많아서 나중에 profiling 할 때 시간이 너무 오래 걸림(30분 넘어가고 빡쳐서 종료함). 
데이터가 충분히 크기 때문에 일부분을 랜덤하게 추출하여 사용.

In [ ]:
#원 데이터의 20%만 사용하기. random_state 를 변경해주면 뽑는 데이터가 랜덤하게 바뀜. 나는 23을 좋아하니까 23으로 하겠음.
orders = orders.loc[orders.user_id.isin(orders.user_id.drop_duplicates().sample(frac=0.20, random_state=23))] 

In [ ]:
# Convert character variables into category. 
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

# **Feature 설정**

6. 장바구니에 담긴 순서의 평균
8. 장바구니에 담고 구매한 비율
9. 장바구니에 담고 구매하지 않은 비율

1. 유저의 재주문 횟수
2. 유저의 재주문율
3. 유저의 물건별 재주문율
4. 물건 재주문횟수
5. 물건 재주문율
6. 장바구니에 담은 평균 횟수
7. 

# 1, 2 유저의 재주문 횟수, 재주문율

1. 유저의 재주문 횟수, 유저의 물건별 재주문율, 전체 재주문율

In [ ]:
prior_orders = pd.merge(orders, order_products_prior, on='order_id', how='inner')
prior_orders.head(10)

orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
45,2717275,5,prior,1,3,12,NaN
46,1909121,5,prior,2,0,16,11.0
47,2267326,5,prior,3,3,18,10.0
48,157374,5,prior,4,1,18,19.0
49,2196797,5,train,5,0,11,6.0


In [ ]:
# id 당 총 주문 횟수 구하기..
total_orders = prior_orders.groupby('user_id')['order_number'].max().to_frame('total_orders').reset_index()
total_orders.head()

,user_id,total_orders
0,7,17
1,36,10
2,90,31
3,140,72
4,164,5


In [ ]:
#처음 구매했을 때 오더 넘버 구하기..
product_first_order_num = prior_orders.groupby(by=['user_id', 'product_id'])['order_number'].aggregate('min').to_frame('first_order_number').reset_index()
product_first_order_num.head()

,user_id,product_id,first_order_number
0,7,4920,17
1,7,4945,17
2,7,8277,17
3,7,11520,17
4,7,13198,17


In [ ]:
# total order 랑 첫 주문 합치기..
user_product_df = pd.merge(total_orders, product_first_order_num, on='user_id', how='right')
user_product_df.head()

,user_id,total_orders,product_id,first_order_number
0,7,17,4920,17
1,7,17,4945,17
2,7,17,8277,17
3,7,17,11520,17
4,7,17,13198,17


In [ ]:
# Calculate the order range.
# The +1 includes in the difference is the first order where the product has been purchased
user_product_df['order_range'] = user_product_df['total_orders'] - user_product_df['first_order_number'] + 1
user_product_df.head()

,user_id,total_orders,product_id,first_order_number,order_range
0,7,17,4920,17,1
1,7,17,4945,17,1
2,7,17,8277,17,1
3,7,17,11520,17,1
4,7,17,13198,17,1


In [ ]:
#Create  a dataframe to show the number of times a user have bough a product.
number_of_times = prior_orders.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('times_bought').reset_index()
number_of_times.head()

,user_id,product_id,times_bought
0,7,4920,1
1,7,4945,1
2,7,8277,1
3,7,11520,1
4,7,13198,1


In [ ]:
# Merging number_of_times with user_product_df
uxp_ratio = pd.merge(number_of_times, user_product_df, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range
0,7,4920,1,17,17,1
1,7,4945,1,17,17,1
2,7,8277,1,17,17,1
3,7,11520,1,17,17,1
4,7,13198,1,17,17,1


In [ ]:
# Get a dataframe to calculate the reorder ratio for each product
uxp_ratio['users_product_reorder_ratio'] = uxp_ratio['times_bought'] / uxp_ratio['order_range']
uxp_ratio.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio
0,7,4920,1,17,17,1,1.0
1,7,4945,1,17,17,1,1.0
2,7,8277,1,17,17,1,1.0
3,7,11520,1,17,17,1,1.0
4,7,13198,1,17,17,1,1.0


In [ ]:
#재주문 비율을 구하기 위해 선 작업..
user_reorder_ratio = prior_orders.groupby(by='user_id')['reordered'].aggregate('mean').to_frame('reorder_ratio').reset_index()
user_reorder_ratio['reorder_ratio'] = user_reorder_ratio['reorder_ratio'].astype(np.float16)
user_reorder_ratio.head()

,user_id,reorder_ratio
0,7,0.818359
1,36,0.333252
2,90,0.833496
3,140,0.913086
4,164,0.959961


In [ ]:
# user_product_df랑 merge
uxp_ratio = pd.merge(uxp_ratio, user_reorder_ratio, on=['user_id'], how='left')
uxp_ratio.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio
0,7,4920,1,17,17,1,1.0,0.818359
1,7,4945,1,17,17,1,1.0,0.818359
2,7,8277,1,17,17,1,1.0,0.818359
3,7,11520,1,17,17,1,1.0,0.818359
4,7,13198,1,17,17,1,1.0,0.818359


# **3,4 물건의 재주문 횟수, 재주문율**

3. 물건의 재주문 횟수

In [ ]:
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,23391,7,prior,17,0,10,28.0,13198,1.0,1.0
1,23391,7,prior,17,0,10,28.0,42803,2.0,1.0
2,23391,7,prior,17,0,10,28.0,8277,3.0,1.0
3,23391,7,prior,17,0,10,28.0,37602,4.0,1.0
4,23391,7,prior,17,0,10,28.0,40852,5.0,1.0


In [ ]:
# 물건 재 주문 횟수 구하기.. count
purchased_num_of_times = prior_orders.groupby(by='product_id')['order_id'].aggregate('count').to_frame('purchased_num_of_times').reset_index()
purchased_num_of_times.head()

,product_id,purchased_num_of_times
0,44,1
1,45,2
2,54,1
3,148,1
4,196,1


In [ ]:
purchased_num_of_times = pd.merge(uxp_ratio, purchased_num_of_times, on='product_id', how='left')

In [ ]:
purchased_num_of_times.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times
0,7,4920,1,17,17,1,1.0,0.818359,8
1,7,4945,1,17,17,1,1.0,0.818359,1
2,7,8277,1,17,17,1,1.0,0.818359,9
3,7,11520,1,17,17,1,1.0,0.818359,7
4,7,13198,1,17,17,1,1.0,0.818359,2


In [ ]:
#각각의 상품 재주문율 구하기
product_reorder_ratio = prior_orders.groupby(by='product_id')['reordered'].aggregate('mean').to_frame('product_reorder_ratio').reset_index()
product_reorder_ratio['product_reorder_ratio'] = product_reorder_ratio['product_reorder_ratio'].astype(np.float16)
product_reorder_ratio.head()

,product_id,product_reorder_ratio
0,44,1.0
1,45,1.0
2,54,0.0
3,148,0.0
4,196,1.0


In [ ]:
#장바구니에 담긴 평균 횟수,,
add_to_cart = prior_orders.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').to_frame('product_avg_cart_addition').reset_index()
add_to_cart.head()

,product_id,product_avg_cart_addition
0,44,1.0
1,45,4.0
2,54,3.0
3,148,15.0
4,196,4.0


In [ ]:
purchased_num_of_times.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times
0,7,4920,1,17,17,1,1.0,0.818359,8
1,7,4945,1,17,17,1,1.0,0.818359,1
2,7,8277,1,17,17,1,1.0,0.818359,9
3,7,11520,1,17,17,1,1.0,0.818359,7
4,7,13198,1,17,17,1,1.0,0.818359,2


In [ ]:
# 지금까지 구한거 merge
purchased_num_of_times = purchased_num_of_times.merge(product_reorder_ratio, on='product_id', how='left')
purchased_num_of_times = purchased_num_of_times.merge(add_to_cart, on='product_id', how='left')


In [ ]:
purchased_num_of_times.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times,product_reorder_ratio,product_avg_cart_addition
0,7,4920,1,17,17,1,1.0,0.818359,8,0.750000,8.500000
1,7,4945,1,17,17,1,1.0,0.818359,1,1.000000,7.000000
2,7,8277,1,17,17,1,1.0,0.818359,9,0.666504,9.222222
3,7,11520,1,17,17,1,1.0,0.818359,7,0.571289,5.857143
4,7,13198,1,17,17,1,1.0,0.818359,2,1.000000,3.500000


In [ ]:
#장바구니에 담고 구매한 비율 cart_in_order 구하기
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,23391,7,prior,17,0,10,28.0,13198,1.0,1.0
1,23391,7,prior,17,0,10,28.0,42803,2.0,1.0
2,23391,7,prior,17,0,10,28.0,8277,3.0,1.0
3,23391,7,prior,17,0,10,28.0,37602,4.0,1.0
4,23391,7,prior,17,0,10,28.0,40852,5.0,1.0


In [ ]:
add_to_cart_count = prior_orders.groupby(by='product_id')['add_to_cart_order'].aggregate('count').to_frame('product_avg_cart_addition').reset_index()
add_to_cart_count.head()

,product_id,product_avg_cart_addition
0,44,1
1,45,2
2,54,1
3,148,1
4,196,1


In [ ]:
A = prior_orders[prior_orders.reordered==1]
A.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,23391,7,prior,17,0,10,28.0,13198,1.0,1.0
1,23391,7,prior,17,0,10,28.0,42803,2.0,1.0
2,23391,7,prior,17,0,10,28.0,8277,3.0,1.0
3,23391,7,prior,17,0,10,28.0,37602,4.0,1.0
4,23391,7,prior,17,0,10,28.0,40852,5.0,1.0


In [ ]:

add_to_cart_count_re = A.groupby(by='product_id')['add_to_cart_order'].aggregate('count').to_frame('product_avg_cart_addition_re').reset_index()
add_to_cart_count_re.head()

,product_id,product_avg_cart_addition_re
0,44,1
1,45,2
2,196,1
3,258,1
4,267,1


In [ ]:
purchased_num_of_times['cart_in_order'] = add_to_cart_count_re['product_avg_cart_addition_re'] / add_to_cart_count['product_avg_cart_addition']

In [ ]:
purchased_num_of_times.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times,product_reorder_ratio,product_avg_cart_addition,cart_in_order
0,6,10644,1,2.0,2.0,1.0,1.0,0.285645,442,0.414062,9.819005,0.666667
1,6,11068,1,2.0,2.0,1.0,1.0,0.285645,380,0.631348,9.647368,0.500000
2,6,20323,1,2.0,2.0,1.0,1.0,0.285645,92,0.521973,9.652174,3.333333
3,6,21903,1,2.0,2.0,1.0,1.0,0.285645,8241,0.767090,7.371314,0.285714
4,6,38293,1,2.0,2.0,1.0,1.0,0.285645,893,0.645996,8.650616,0.250000


In [ ]:
#장바구니에 담고 구매하지 않은 비율 cart_in_not_order 구하기
purchased_num_of_times['cart_in_order'] = purchased_num_of_times['cart_in_order']*purchased_num_of_times['product_reorder_ratio']

In [ ]:
purchased_num_of_times.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times,product_reorder_ratio,product_avg_cart_addition,cart_in_order
0,6,10644,1,2.0,2.0,1.0,1.0,0.285645,442,0.414062,9.819005,0.276042
1,6,11068,1,2.0,2.0,1.0,1.0,0.285645,380,0.631348,9.647368,0.315674
2,6,20323,1,2.0,2.0,1.0,1.0,0.285645,92,0.521973,9.652174,1.739909
3,6,21903,1,2.0,2.0,1.0,1.0,0.285645,8241,0.767090,7.371314,0.219169
4,6,38293,1,2.0,2.0,1.0,1.0,0.285645,893,0.645996,8.650616,0.161499


In [ ]:
B = prior_orders[prior_orders.reordered==0]
B.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
1,298250,6,prior,2.0,4.0,16.0,6.0,20323,2.0,0.0
2,298250,6,prior,2.0,4.0,16.0,6.0,40992,3.0,0.0
4,298250,6,prior,2.0,4.0,16.0,6.0,45007,5.0,0.0
5,298250,6,prior,2.0,4.0,16.0,6.0,11068,6.0,0.0
6,298250,6,prior,2.0,4.0,16.0,6.0,10644,7.0,0.0


In [ ]:
add_to_cart_count_not = A.groupby(by='product_id')['add_to_cart_order'].aggregate('count').to_frame('product_avg_cart_addition_not').reset_index()
add_to_cart_count_not.head()

,product_id,product_avg_cart_addition_not
0,1,54
1,3,2
2,4,10
3,5,4
4,8,1


In [ ]:
purchased_num_of_times['cart_in_not_order'] = add_to_cart_count_not['product_avg_cart_addition_not'] / add_to_cart_count['product_avg_cart_addition']

In [ ]:
purchased_num_of_times.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times,product_reorder_ratio,product_avg_cart_addition,cart_in_order,cart_in_not_order
0,6,10644,1,2.0,2.0,1.0,1.0,0.285645,442,0.414062,9.819005,0.276042,0.666667
1,6,11068,1,2.0,2.0,1.0,1.0,0.285645,380,0.631348,9.647368,0.315674,0.500000
2,6,20323,1,2.0,2.0,1.0,1.0,0.285645,92,0.521973,9.652174,1.739909,3.333333
3,6,21903,1,2.0,2.0,1.0,1.0,0.285645,8241,0.767090,7.371314,0.219169,0.285714
4,6,38293,1,2.0,2.0,1.0,1.0,0.285645,893,0.645996,8.650616,0.161499,0.250000


In [ ]:
purchased_num_of_times['cart_in_not_order'] = purchased_num_of_times['cart_in_not_order']*(1-purchased_num_of_times['product_reorder_ratio'])

In [ ]:
purchased_num_of_times.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times,product_reorder_ratio,product_avg_cart_addition,cart_in_order,cart_in_not_order
0,7,4920,1,17,17,1,1.0,0.818359,8,0.750000,8.500000,0.750000,0.250000
1,7,4945,1,17,17,1,1.0,0.818359,1,1.000000,7.000000,1.000000,0.000000
2,7,8277,1,17,17,1,1.0,0.818359,9,0.666504,9.222222,0.666504,0.333496
3,7,11520,1,17,17,1,1.0,0.818359,7,0.571289,5.857143,0.571289,0.428711
4,7,13198,1,17,17,1,1.0,0.818359,2,1.000000,3.500000,1.000000,0.000000


In [ ]:
#파이널 데이터로 데이터 넘겨받기..
final_data = purchased_num_of_times.copy()

In [ ]:
final_data.head()

,user_id,product_id,times_bought,total_orders,first_order_number,order_range,users_product_reorder_ratio,reorder_ratio,purchased_num_of_times,product_reorder_ratio,product_avg_cart_addition,cart_in_order,cart_in_not_order
0,7,4920,1,17,17,1,1.0,0.818359,8,0.750000,8.500000,0.750000,0.250000
1,7,4945,1,17,17,1,1.0,0.818359,1,1.000000,7.000000,1.000000,0.000000
2,7,8277,1,17,17,1,1.0,0.818359,9,0.666504,9.222222,0.666504,0.333496
3,7,11520,1,17,17,1,1.0,0.818359,7,0.571289,5.857143,0.571289,0.428711
4,7,13198,1,17,17,1,1.0,0.818359,2,1.000000,3.500000,1.000000,0.000000


In [ ]:
!pip install -U pandas-profiling

     |████████████████████████████████| 261 kB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 46.3 MB/s 
     |████████████████████████████████| 10.1 MB 42.5 MB/s 
     |████████████████████████████████| 102 kB 9.9 MB/s 
     |████████████████████████████████| 62 kB 621 kB/s 
     |████████████████████████████████| 675 kB 64.2 MB/s 
     |████████████████████████████████| 636 kB 58.6 MB/s 
     |████████████████████████████████| 812 kB 56.0 MB/s 
     |████████████████████████████████| 28.5 MB 44 kB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=299cace90ba319bac0dd557a33d6693f3e6b1afda43725147d2fd4682fa8bae6
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=ff389ab080bacc1d606f88b4f9ff4a816385693c5359f638459dae85cd72d737
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e2

In [ ]:
final_data.profile_report()

In [ ]:
!pip install lightgbm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
lgbm.fit(train_X, train_Y)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc
sns.set_style("whitegrid")

NameError: ignored

     |████████████████████████████████| 261 kB 5.2 MB/s 
     |████████████████████████████████| 102 kB 10.6 MB/s 
     |████████████████████████████████| 636 kB 50.3 MB/s 
     |████████████████████████████████| 3.1 MB 28.6 MB/s 
     |████████████████████████████████| 675 kB 33.6 MB/s 
     |████████████████████████████████| 62 kB 742 kB/s 
     |████████████████████████████████| 10.1 MB 56.1 MB/s 
     |████████████████████████████████| 812 kB 43.1 MB/s 
     |████████████████████████████████| 28.5 MB 50 kB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=d24bc53009efe8b4d2614c73cc36040b6faedcdbce49b8367afbd4042d52047e
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=60466fef04da59fb4e69d4ad9e84b454f4ba567bcf01b176eaf1b36fb3c68186
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e

In [ ]:
import pandas_profiling

In [ ]:
final_data.profile_report()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
train_df, test_df = train_test_split(df, test_size = 0.3, random_state= 42)

NameError: ignored